In [1]:
import os
import time

import re

import openai
os.environ['OPENAI_API_KEY'] = "EMPTY"
os.environ['OPENAI_API_BASE'] = "http://10.0.0.222:30307/v1"
openai.api_key = "EMPTY"
openai.api_base = "http://10.0.0.222:30307/v1"

model = "Writer/camel-5b-hf"
#model = "mosaicml/mpt-7b-instruct"
#model = "mosaicml/mpt-30b-instruct"

In [2]:
import logging
import sys

#kron extensions to llama_index to support openai compatible api
sys.path.append('../llama_index')

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [3]:
from datetime import datetime
import json
import pathlib
import fitz

PDF_BASE = '/home/arylwen/datasets/documents/pdf/'
TXT_BASE = '/home/arylwen/datasets/documents/text_cleaned/'
JSON_BASE = '/home/arylwen/datasets/documents/json_cleaned/'
doc_path = '/home/arylwen/datasets/documents/json_raw/2007.02871.pdf.json'

In [4]:
def read_text_file(filename):
    json_content = pathlib.Path(filename).read_bytes()
    print(len(json_content))
    return json_content

doc_string = read_text_file(doc_path) #.decode(encoding = 'utf-8')
doc_string = doc_string.decode(encoding = 'utf-8')
print(doc_string)

55211
{"title": "2007.02871.pdf", "text": "DART: Open-Domain Structured Data Record to Text Generation\nLinyong Nan1\nDragomir Radev1,2\nRui Zhang3\nAmrit Rau1\nAbhinand Sivaprasad1\nChiachun Hsieh4\nXiangru Tang1\nAadit Vyas1\nNeha Verma1\nPranav Krishna5\nYangxiaokang Liu1\nNadia Irwanto1\nJessica Pan1\nFaiaz Rahman1\nAhmad Zaidi1\nMurori Mutuma1\nYasin Tarabar1\nAnkit Gupta1\nTao Yu1\nYi Chern Tan1\nXi Victoria Lin2\u2217\nCaiming Xiong2\nRichard Socher2\nNazneen Fatema Rajani2\n1 Yale University\n2 Salesforce Research\n3 Penn State University\n4 The University of Hong Kong\n5 MIT\n{linyong.nan, dragomir.radev}@yale.edu, rmz5227@psu.edu, nazneen.rajani@salesforce.com\nAbstract\nWe present DART, an open domain structured\nDAta Record to Text generation dataset with\nover 82k instances (DARTs). Data-to-Text an-\nnotations can be a costly process, especially\nwhen dealing with tables which are the ma-\njor source of structured data and contain non-\ntrivial structures. To this end, we 

In [5]:
document = json.loads(doc_string)

In [6]:
document

{'title': '2007.02871.pdf',
 'text': 'DART: Open-Domain Structured Data Record to Text Generation\nLinyong Nan1\nDragomir Radev1,2\nRui Zhang3\nAmrit Rau1\nAbhinand Sivaprasad1\nChiachun Hsieh4\nXiangru Tang1\nAadit Vyas1\nNeha Verma1\nPranav Krishna5\nYangxiaokang Liu1\nNadia Irwanto1\nJessica Pan1\nFaiaz Rahman1\nAhmad Zaidi1\nMurori Mutuma1\nYasin Tarabar1\nAnkit Gupta1\nTao Yu1\nYi Chern Tan1\nXi Victoria Lin2∗\nCaiming Xiong2\nRichard Socher2\nNazneen Fatema Rajani2\n1 Yale University\n2 Salesforce Research\n3 Penn State University\n4 The University of Hong Kong\n5 MIT\n{linyong.nan, dragomir.radev}@yale.edu, rmz5227@psu.edu, nazneen.rajani@salesforce.com\nAbstract\nWe present DART, an open domain structured\nDAta Record to Text generation dataset with\nover 82k instances (DARTs). Data-to-Text an-\nnotations can be a costly process, especially\nwhen dealing with tables which are the ma-\njor source of structured data and contain non-\ntrivial structures. To this end, we propose a\

In [7]:
text = document['text']
text

'DART: Open-Domain Structured Data Record to Text Generation\nLinyong Nan1\nDragomir Radev1,2\nRui Zhang3\nAmrit Rau1\nAbhinand Sivaprasad1\nChiachun Hsieh4\nXiangru Tang1\nAadit Vyas1\nNeha Verma1\nPranav Krishna5\nYangxiaokang Liu1\nNadia Irwanto1\nJessica Pan1\nFaiaz Rahman1\nAhmad Zaidi1\nMurori Mutuma1\nYasin Tarabar1\nAnkit Gupta1\nTao Yu1\nYi Chern Tan1\nXi Victoria Lin2∗\nCaiming Xiong2\nRichard Socher2\nNazneen Fatema Rajani2\n1 Yale University\n2 Salesforce Research\n3 Penn State University\n4 The University of Hong Kong\n5 MIT\n{linyong.nan, dragomir.radev}@yale.edu, rmz5227@psu.edu, nazneen.rajani@salesforce.com\nAbstract\nWe present DART, an open domain structured\nDAta Record to Text generation dataset with\nover 82k instances (DARTs). Data-to-Text an-\nnotations can be a costly process, especially\nwhen dealing with tables which are the ma-\njor source of structured data and contain non-\ntrivial structures. To this end, we propose a\nprocedure of extracting semantic tri

In [8]:
#convert split words on line break, e.g. post-\nediting
text = text.replace('-\n', '')
#remove new lines in the middle of the sentence
one_new_line = r'(?<![\.\n])\n(?!\n)'
text = re.sub(one_new_line, ' ', text)

In [9]:
document['text'] = text
text

'DART: Open-Domain Structured Data Record to Text Generation Linyong Nan1 Dragomir Radev1,2 Rui Zhang3 Amrit Rau1 Abhinand Sivaprasad1 Chiachun Hsieh4 Xiangru Tang1 Aadit Vyas1 Neha Verma1 Pranav Krishna5 Yangxiaokang Liu1 Nadia Irwanto1 Jessica Pan1 Faiaz Rahman1 Ahmad Zaidi1 Murori Mutuma1 Yasin Tarabar1 Ankit Gupta1 Tao Yu1 Yi Chern Tan1 Xi Victoria Lin2∗ Caiming Xiong2 Richard Socher2 Nazneen Fatema Rajani2 1 Yale University 2 Salesforce Research 3 Penn State University 4 The University of Hong Kong 5 MIT {linyong.nan, dragomir.radev}@yale.edu, rmz5227@psu.edu, nazneen.rajani@salesforce.com Abstract We present DART, an open domain structured DAta Record to Text generation dataset with over 82k instances (DARTs). Data-to-Text annotations can be a costly process, especially when dealing with tables which are the major source of structured data and contain nontrivial structures. To this end, we propose a procedure of extracting semantic triples from tables that encodes their structure

In [10]:
def write_text_file(filename, content):
    pathlib.Path(TXT_BASE+filename).write_bytes(content.encode('utf-8').strip())

def write_json_file(filename, content):
    pathlib.Path(JSON_BASE+filename).write_bytes(content.encode('utf-8').strip())

def save_cleaned_file(document):
    filename = document['title']+'.json'
    filename_txt = document['title']+'.txt'
    json_object = json.dumps(document) 
    write_json_file(filename,json_object)
    write_text_file(filename_txt,document['text'])

In [11]:
save_cleaned_file(document)